Script for inserting weather station list to the database.

In [1]:
import csv
import psycopg2

In [2]:
csv_file = "../data/weather_station_list/data/weather_sta_list_ENG.csv" 
table_name = "stations"
conn_string = "postgresql://lizavabistsevits:@localhost:5432/taiwan" # database connection string

column_names = {
    "Code": "code",
    "Name": "name",
    "Original_Name" : "orig_name",
    "Type": "type",
    "Latitude": "latitude",
    "Longitude": "longitude",
	"Altitude" : "altitude",
	"Data_Start_Date" : "data_start_date",
	"Data_End_Date" : "data_end_date"
}

In [3]:
"""
with open(csv_file, 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Create dictionary for each row
    for row in csv_reader:
        print(row)
"""


"\nwith open(csv_file, 'r') as file:\n    csv_reader = csv.DictReader(file)\n    \n    # Create dictionary for each row\n    for row in csv_reader:\n        print(row)\n"

In [4]:
# Connect to database
with psycopg2.connect(conn_string) as conn:
    with conn.cursor() as cursor:

        # Create a set to record processed station codes
        processed_codes = set()
        duplicates = 0
        
        with open(csv_file, 'r') as file:
            csv_reader = csv.DictReader(file)
            
            # Insert data row by row
            for row in csv_reader:
                # Check for duplicate station codes
                code_value = row["Code"]
                if code_value in processed_codes:
                    duplicates += 1
                    continue
                # Add to the tracking set
                processed_codes.add(code_value)
                
                # Prepare column names and values for the specific columns
                columns = []
                values = []
                
                for csv_col, db_col in column_names.items():
                    if csv_col in row:
                        columns.append(db_col)
                        # Convert empty strings to NULL
                        if row[csv_col] == "":
                            values.append(None)
                        else:
                            values.append(row[csv_col])
                
                # Create the INSERT query string with the specified columns
                columns_joined = ", ".join(columns)
                placeholders = ", ".join(["%s"] * len(values))
                insert_query = f"INSERT INTO {table_name} ({columns_joined}) VALUES ({placeholders});"
                
                # Execute the query
                cursor.execute(insert_query, values)
        
        # Commit changes
        conn.commit()

print(f"Data from {csv_file} successfully inserted to {table_name} table.")
print(f"Processed {len(processed_codes)} unique stations and skipped {duplicates} duplicates.")

Data from ../data/weather_station_list/data/weather_sta_list_ENG.csv successfully inserted to stations table.
Processed 1155 unique stations and skipped 80 duplicates.
